# Multi Head Attention

In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))

# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("This notebook use [%s]."%(device))

PyTorch version:[1.7.0].
This notebook use [cuda:0].


# Set hyper-parameter

In [2]:
EPOCHS = 10
FEATURES=28
NUM_HEADS=4
BATCH_SIZE = 256
LEARNING_RATE = 0.001

# Prepare dataset

In [3]:
# MNIST dataset
mnist_train = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=True)

train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

print("Preparing dataset done!")

Preparing dataset done!


# Defining Model



![image](./figures/MHA/fig02.png)

## Scaled Dot-Product Attention

$$

Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V

$$

In [4]:
class ScaledDotProductAttention(nn.Module):

    def forward(self, query, key, value, mask=None):
        dk = key.size()[-1]
        scores = query.matmul(key.transpose(-2, -1)) / np.sqrt(dk)
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)  
        attention = torch.softmax(scores, dim=-1)
        out = attention.matmul(value)
        return out, attention

y = torch.rand(1, 28, 28)
out = ScaledDotProductAttention()(y, y, y)
out[0].shape, out[1].shape

(torch.Size([1, 28, 28]), torch.Size([1, 28, 28]))

## Multi Head Attention



In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, features, num_heads, bias=True, dropout_ratio = 0.1, activation=F.relu_):
        super(MultiHeadAttention, self).__init__()
        assert features % num_heads == 0, f'"features"(features) should be divisible by "head_num"(num_heads)'
        
        self.features = features
        self.num_heads = num_heads
        self.head_features = features // num_heads
        self.bias = bias
        self.dropout_ratio = dropout_ratio
        self.act = activation

        self.wq = nn.Linear(features, features, bias=bias)
        self.wk = nn.Linear(features, features, bias=bias)
        self.wv = nn.Linear(features, features, bias=bias)

        self.fc = nn.Linear(features, features, bias=bias)
        
        self.dropout = nn.Dropout(dropout_ratio)
        
    def split_heads(self, x, batch_size):
        # batch_size, num_heads, seq_len, head_features
        x = x.reshape(batch_size, -1, self.num_heads, self.head_features)
        return x.permute([0, 2, 1, 3])
    
    def forward(self, q, k, v, mask=None):
        # q, k, v: (batch_size, seq_len, features)
        batch_size = q.size(0)

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = ScaledDotProductAttention()(q, k, v, mask)

        scaled_attention = scaled_attention.permute([0, 2, 1, 3])

        scaled_attention = scaled_attention.reshape(batch_size, -1, self.features)

        out = self.fc(scaled_attention)
        if self.act is not None:
            out = self.act(out)

        return out, attention_weights 

## Transformer Encoder

![image](./figures/ViT/fig01.png)

In [6]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, features, fffeatures):
        super(FeedForwardNetwork, self).__init__()

        layer_list = [
            nn.Linear(features, fffeatures),
            nn.ReLU(True),
            nn.Linear(fffeatures, features)
        ]

        self.net = nn.Sequential(*layer_list)
    def forward(self, x):
        return self.net(x)

class EncoderLayer(nn.Module):
    def __init__(self, features, num_heads, fffeatures, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(features, num_heads)
        self.ffn = FeedForwardNetwork(features, fffeatures)

        self.attnlayernorm = nn.LayerNorm(features)
        self.ffnlayernorm = nn.LayerNorm(features)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, mask):

        attn_output, _ = self.mha(x, x, x, mask)  
        attn_output = self.dropout(attn_output)
        out = self.attnlayernorm(x + attn_output)  

        ffn_output = self.ffn(out)  
        ffn_output = self.dropout(ffn_output)
        out = self.ffnlayernorm(out + ffn_output)  

        return out

In [12]:
# our model
class Model(nn.Module):
    def __init__(self, features=28, num_heads=4, num_layers=6, fffeatures=512, dropout_ratio=0.1, num_classes=10, init_weight="he", init_bias="zero"):
        super(Model, self).__init__()

        self.init_weight = init_weight
        self.init_bias = init_bias
        
        
        self.transformer_encoder = nn.Sequential(*[EncoderLayer(features, num_heads, fffeatures, dropout_ratio) for _ in range(num_layers)])
        self.linear = nn.Linear(fffeatures, num_classes)

        self.init_params()
        
    def init_params(self):
        
        init_weight_method = {
        "he": nn.init.kaiming_normal_, 
        "xavier": nn.init.xavier_normal_
        }
        assert self.init_weight in init_weight_method.keys(), f'Select the weight initialization method in {list(init_weight_method.keys())}'
        
        init_bias_method = {
            "zero": nn.init.zeros_, 
            "uniform": nn.init.uniform_
        }
        assert self.init_bias in init_bias_method.keys(), f'Select the bias initialization method in {list(init_bias_method.keys())}'

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weight_method[self.init_weight](m.weight)
                init_bias_method[self.init_bias](m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x, _ = self.mha(x, x, x)
        return self.net(x)

model = Model(FEATURES, NUM_HEADS).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Calculate Parameters

In [14]:
total_params = 0
for param_name, param in model.named_parameters():
    if param.requires_grad:
        total_params += len(param.reshape(-1))
print(f"Number of Total Parameters: {total_params:,d}")

Number of Total Parameters: 200,562


# Define help function

In [ ]:
def test_eval(model, data_iter, batch_size):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        total = 0
        correct = 0
        for batch_img, batch_lab in data_iter:
            X = batch_img.view(-1, 1, 28, 28).to(device)
            Y = batch_lab.to(device)
            y_pred = model(X)
            _, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == Y).sum().item()
            total += batch_img.size(0)
        val_acc = (100 * correct / total)
        model.train()
    return val_acc

# Train model

In [ ]:
# Training Phase
print_every = 1
print("Start training !")
# Training loop
for epoch in range(EPOCHS):
    loss_val_sum = 0
    for batch_img, batch_lab in train_iter:

        X = batch_img.view(-1, 1, 28, 28).to(device)
        Y = batch_lab.to(device)
        
        # Inference & Calculate los
        y_pred = model.forward(X)
        loss = criterion(y_pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_val_sum += loss
        
    if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
        # accr_val = M.test(x_test, y_test, batch_size)
        loss_val_avg = loss_val_sum / len(train_iter)
        accr_val = test_eval(model, test_iter, BATCH_SIZE)
        print(f"epoch:[{epoch+1}/{EPOCHS}] cost:[{loss_val_avg:.3f}] test_accuracy:[{accr_val:.3f}]")
print("Training Done !")

# Test model

In [ ]:
n_sample = 64
sample_indices = np.random.choice(len(mnist_test.targets), n_sample, replace=False)
test_x = mnist_test.data[sample_indices]
test_y = mnist_test.targets[sample_indices]

with torch.no_grad():
    model.eval()
    y_pred = model.forward(test_x.view(-1, 1, 28, 28).type(torch.float).to(device))
    model.train()

y_pred = y_pred.argmax(axis=1)

plt.figure(figsize=(20, 20))

for idx in range(n_sample):
    plt.subplot(8, 8, idx+1)
    plt.imshow(test_x[idx], cmap='gray')
    plt.axis('off')
    plt.title(f"Predict: {y_pred[idx]}, Label: {test_y[idx]}")

plt.show()